In [1]:
import mysql.connector
import pandas as pd
import numpy as np
import re

In [2]:
mydb = mysql.connector.connect(
    host="localhost",
    user="student",
    password="student")
print(mydb)

In [3]:
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS proyecto_final")
mycursor.execute("USE proyecto_final")

In [4]:
camposDict = {'arte_diseno_arquitectura':1, 'agricultura_forestal':2, 'empresariales_administracion':3,
             'economicas':4, 'educacion':5, 'ingenieria_tecnologia':6, 'historia_filosofia_teologia':7,
             'leyes_abogacia':8, 'ciencias_salud':9, 'ciencias_naturales': 10, 'ciencias_sociales':11, 'otros':12}

In [5]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Campos (\
                id int PRIMARY KEY, \
                nombre varchar(255))")
sqlCampos = ("INSERT INTO Campos (id, nombre) VALUES (%s, %s)")
val = []
for k, v in camposDict.items():
    val.append((v, k))
mycursor.executemany(sqlCampos, val)
mydb.commit()

In [6]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Universidades (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                nombre varchar(255), \
                url varchar(255), \
                posicion_hindex int, \
                siglas varchar(255), \
                codigo_postal int, \
                ciudad varchar(255), \
                nombre_ingles varchar(255), \
                tipo_centro varchar(255))")

In [7]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Facultades (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                nombre varchar(255), \
                id_univ int, \
                id_campo int, \
                FOREIGN KEY (id_univ) REFERENCES Universidades(id) ON DELETE CASCADE ON UPDATE CASCADE, \
                FOREIGN KEY (id_campo) REFERENCES Campos(id) ON DELETE CASCADE ON UPDATE CASCADE)")

In [8]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Grados (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                id_fac int, \
                nombre varchar(255), \
                presencial bool, \
                nota_corte double, \
                idioma bool, \
                nota_corte_ant double, \
                nota_admision double, \
                rendimiento double, \
                creditos double, \
                precio_creditos double, \
                empleabilidad double, \
                FOREIGN KEY (id_fac) REFERENCES Facultades(id) ON DELETE CASCADE ON UPDATE CASCADE)")

In [9]:
mycursor.execute("CREATE TABLE IF NOT EXISTS Rankings (\
                id int AUTO_INCREMENT PRIMARY KEY, \
                id_univ int, \
                id_campo int, \
                puntuacion int, \
                FOREIGN KEY (id_univ) REFERENCES Universidades(id) ON DELETE CASCADE ON UPDATE CASCADE, \
                FOREIGN KEY (id_campo) REFERENCES Campos(id) ON DELETE CASCADE ON UPDATE CASCADE)")

In [10]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('Campos',)
('Facultades',)
('Grados',)
('Rankings',)
('Universidades',)


In [11]:
DATOS_QEDU_PATH = '../Datos/Datos_QEDU.csv'
UNI_RANKINGS_PATH = '../Datos/uni_rankings_final.csv'
df = pd.read_csv(DATOS_QEDU_PATH).replace(to_replace='\(.+\)', value = '', regex = True)\
    .drop_duplicates(subset=['NOMBRE_UNIV', 'NOMBRE_CENTRO', 'NOM_TITULA']).reset_index()
dfRank = pd.read_csv(UNI_RANKINGS_PATH)
df['NOMBRE_UNIV']=df['NOMBRE_UNIV'].str.strip()
dfRank['nombre_regional']=dfRank['nombre_regional'].str.strip()
dfRank.rename(columns={'nombre_regional':'NOMBRE_UNIV'}, inplace=True)
dfNoneRank = dfRank.replace([-1], [None])
df_join=dfRank.merge(df, on ='NOMBRE_UNIV', how='inner')
df_join

,nombre,nombre_ingles,NOMBRE_UNIV,siglas,ciudad,codigo_postal,posicion_hindex,agricultura_forestal,arte_diseno_arquitectura,empresariales_administracion,...,PRESENCIALIDAD,IDIOMAEXTRANJERO,PLAZAS,NOTA_CORTE,NOTA_CORTE_ANT,NOTA_ADMISION,RENDIMIENTO,CREDITOS,PRECIO,TASA_AFIL
0,Universidad Nacional de Educación a Distancia,National University of Distance Education,Universidad Nacional de Educación a Distancia,UNED,Madrid,28015,57,0,1,1,...,No,No,NaN,NaN,NaN,NaN,59.0,240.0,20.48,75.8
1,Universidad Nacional de Educación a Distancia,National University of Distance Education,Universidad Nacional de Educación a Distancia,UNED,Madrid,28015,57,0,1,1,...,No,No,NaN,NaN,NaN,NaN,40.0,240.0,21.60,NaN
2,Universidad Nacional de Educación a Distancia,National University of Distance Education,Universidad Nacional de Educación a Distancia,UNED,Madrid,28015,57,0,1,1,...,No,No,NaN,NaN,NaN,NaN,70.0,240.0,15.95,NaN
3,Universidad Nacional de Educación a Distancia,National University of Distance Education,Universidad Nacional de Educación a Distancia,UNED,Madrid,28015,57,0,1,1,...,No,No,NaN,NaN,NaN,NaN,48.0,240.0,13.20,78.8
4,Universidad Nacional de Educación a Distancia,National University of Distance Education,Universidad Nacional de Educación a Distancia,UNED,Madrid,28015,57,0,1,1,...,No,No,NaN,NaN,NaN,NaN,26.0,240.0,21.60,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3565,Universidad Internacional de la Empresa,International University of Bussiness,Universidad Internacional de la Empresa,UNIE,Madrid,28036,-1,-1,-1,-1,...,Sí,Sí*,NaN,NaN,NaN,NaN,NaN,240.0,NaN,NaN
3566,Universidad Vitoria-Gasteiz,University Vitoria-Gasteiz,Universidad Euneiz,EUNEIZ,Álava,1013,-1,-1,-1,-1,...,Sí,No,NaN,NaN,NaN,NaN,NaN,240.0,NaN,NaN
3567,Universidad Vitoria-Gasteiz,University Vitoria-Gasteiz,Universidad Euneiz,EUNEIZ,Álava,1013,-1,-1,-1,-1,...,Sí,No,NaN,NaN,NaN,NaN,NaN,240.0,NaN,NaN
3568,Universidad Vitoria-Gasteiz,University Vitoria-Gasteiz,Universidad Euneiz,EUNEIZ,Álava,1013,-1,-1,-1,-1,...,Sí,No,NaN,NaN,NaN,NaN,NaN,240.0,NaN,NaN


In [12]:
df2 = df_join.replace(['Pública / Propio', 'Pública / Adscrito**'], 'Pública').replace(['Privada / Propio', 'Privada / Adscrito**'], 'Privada')
dfUniv = df2.groupby(['NOMBRE_UNIV', 'URL_INFO', 'TIPO_UNIV_CENTRO', 'siglas', 'ciudad', 'codigo_postal', 'posicion_hindex', 'nombre_ingles'], dropna=False).size().reset_index().iloc[:, :-1].fillna(np.nan).replace([np.nan], [None])
dfUniv

,NOMBRE_UNIV,URL_INFO,TIPO_UNIV_CENTRO,siglas,ciudad,codigo_postal,posicion_hindex,nombre_ingles
0,Atlántico Medio,https://www.universidadatlanticomedio.es/plane...,Privada,UNAM,Las Palmas,None,125,Middle Atlantic
1,CUNEF Universidad,https://www.cunef.edu/estudios/cunef-grado/,Privada,CUNEF,Madrid,28040,-1,CUNEF University
2,ESIC Universidad,https://www.esic.edu/university/grados,Privada,ESIC,Pozuelo de Alarcón,28223,85,ESIC University
3,Hesperides,https://hesperides.edu.es/estudios/,Privada,ULH,None,None,129,Hesperides
4,IE Universidad,http://www.ie.edu/es/universidad/estudios/ofer...,Privada,IEU,Segovia,40003,-1,IE University
...,...,...,...,...,...,...,...,...
83,Universidad de la Rioja,https://www.unirioja.es/estudios/grados/,Pública,URI,Logroño,26006,36,University of La Rioja
84,Universidad de las Illes Balears,http://estudis.uib.cat/es/grau/,Pública,UIB,Palma,7122,13,University of the Balearic Islands
85,Universidad del País Vasco/Euskal Herriko Unib...,https://www.ehu.eus/es/web/estudiosdegrado-gra...,Pública,UPV/EHU,Leioa,48940,17,University of the Basque Country
86,Universitat Abat Oliba CEU,https://www.uaoceu.es/estudios?tipos_estudio=3...,Privada,UAOC,Barcelona,8022,31,Abat Oliba CEU University


In [13]:
sqlUniv = "INSERT INTO Universidades (nombre, url, tipo_centro, posicion_hindex, codigo_postal, ciudad, siglas, nombre_ingles) \
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
for index, row in dfUniv.iterrows():
    val = (row['NOMBRE_UNIV'], row['URL_INFO'], row['TIPO_UNIV_CENTRO'], row['posicion_hindex'], row['codigo_postal'], row['ciudad'], row['siglas'], row['nombre_ingles'])
    mycursor.execute(sqlUniv, val)
mydb.commit()

In [14]:
dfFac = df2.groupby(['NOMBRE_UNIV', 'NOMBRE_CENTRO'], dropna=False).size().reset_index().iloc[:, :-1]
dfFac

,NOMBRE_UNIV,NOMBRE_CENTRO
0,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas
1,CUNEF Universidad,Escuela Politécnica Superior
2,CUNEF Universidad,"Facultad de Empresa, Economía y Derecho"
3,ESIC Universidad,Facultad de Ciencias Jurídicas y Sociales
4,Hesperides,Escuela de Grados
...,...,...
1062,Universitat de València,Facultad de Psicología y Logopedia
1063,Universitat de València,Facultad de Química
1064,Universitat de València,Facultat d'Infermeria i Podologia
1065,Universitat de València,Facultat de Fisioteràpia


In [15]:
campos = []
arte_diseno_arquitectura = ['Arquitectura', 'Diseño', 'Arte']
agricultura_forestal = ['Agricultura', 'Forestal']
empresariales_administracion = ['Empresa', 'Empresariales', 'Relaciones', 'Marketing', 'Comercio']
economicas = ['Económicas', 'Finan']
educacion = ['Educación', 'Enseñanza', 'Magisteri', 'Profesorado']
ingenieria_tecnologia = ['Ingeniería', 'Politécnica', 'Informática', 'Náutica', 'Ingeniero', 'Tecnología']
historia_filosofia_teologia = ['Filosofía', 'Historia']
leyes_abogacia = ['Derecho', 'Jurídicas']
ciencias_salud = ['Psicología', 'Salud', 'Medicina', 'Farmacia', 'Biol', 'Óptica', 'Sociosanitarias',
                  'Alimentación', 'Deporte', 'Odontología', 'Enfermería', 'Fisio', 'Infermeria']
ciencias_sociales = ['Comunicación', 'Humanidades', 'Sociología', 'Humana', 'Turismo', 'Criminología', 'Letras',
                     'Social', 'Filología', 'Lengua', 'Traducción', 'Información' 'Documentación', 'Política']
ciencias_naturales = ['Física', 'Matemáticas', 'Veterinaria', 'Naturaleza', 'Ciencias', 'Bioq',
                      'Geolo', 'Química', 'Estadísti']
for index, row in dfFac.iterrows():
    facultad = row['NOMBRE_CENTRO']
    if any([x in facultad for x in arte_diseno_arquitectura]):
        campos.append('arte_diseno_arquitectura')
    elif any([x in facultad for x in agricultura_forestal]):
        campos.append('agricultura_forestal')
    elif any([x in facultad for x in empresariales_administracion]):
        campos.append('empresariales_administracion')
    elif any([x in facultad for x in economicas]):
        campos.append('economicas')
    elif any([x in facultad for x in educacion]):
        campos.append('educacion')
    elif any([x in facultad for x in ingenieria_tecnologia]):
        campos.append('ingenieria_tecnologia')
    elif any([x in facultad for x in historia_filosofia_teologia]):
        campos.append('historia_filosofia_teologia')
    elif any([x in facultad for x in leyes_abogacia]):
        campos.append('leyes_abogacia')
    elif any([x in facultad for x in ciencias_salud]):
        campos.append('ciencias_salud')
    elif any([x in facultad for x in ciencias_naturales]):
        campos.append('ciencias_naturales')
    elif any([x in facultad for x in ciencias_sociales]):
        campos.append('ciencias_sociales')
    else:
        campos.append('otros')
dfFac2 = dfFac.assign(CAMPOS=campos)

In [16]:
sqlFac = "INSERT INTO Facultades (nombre, id_univ, id_campo) VALUES (%s, %s, %s)"
sqlQueryUniv = "SELECT id FROM Universidades WHERE nombre = %s"
uniAnterior = ""
idUni = -1
for index, row in dfFac2.iterrows():
    uniActual = row['NOMBRE_UNIV']
    if uniAnterior != uniActual:
        mycursor.execute(sqlQueryUniv, (uniActual, ))
        myresult = mycursor.fetchall()
        idUni = myresult[0][0]
        uniAnterior = uniActual
    val = (row['NOMBRE_CENTRO'], idUni, camposDict.get(row['CAMPOS']))
    mycursor.execute(sqlFac, val)
mydb.commit()

In [17]:
dfGrad = df2.groupby(['NOMBRE_UNIV', 'NOMBRE_CENTRO', 'NOM_TITULA', 'PRESENCIALIDAD', 'NOTA_CORTE', 'IDIOMAEXTRANJERO',
                      'NOTA_CORTE_ANT','NOTA_ADMISION', 'RENDIMIENTO',
                      'CREDITOS', 'PRECIO', 'TASA_AFIL'], dropna=False).size().reset_index().iloc[:, :-1]
dfNone= dfGrad.fillna(np.nan).replace([np.nan], [None])
dfNone

,NOMBRE_UNIV,NOMBRE_CENTRO,NOM_TITULA,PRESENCIALIDAD,NOTA_CORTE,IDIOMAEXTRANJERO,NOTA_CORTE_ANT,NOTA_ADMISION,RENDIMIENTO,CREDITOS,PRECIO,TASA_AFIL
0,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Administración y Dirección de Empresas,Sí,None,No,None,None,87.0,240.0,None,None
1,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Cine,Sí,None,No,None,None,None,240.0,None,None
2,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Comunicación,Sí,None,No,None,None,94.0,240.0,None,None
3,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Derecho,Varias modalidades*,None,No,None,None,23.0,240.0,None,None
4,Atlántico Medio,Facultad de Ciencias Sociales y Jurídicas,Grado en Maestro en Educación Infantil,Varias modalidades*,None,No,None,None,96.0,240.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3565,Universitat de València,Facultat d'Infermeria i Podologia,Grado en Enfermería,Sí,12.35,No,12.51,None,90.0,240.0,18.0,78.3
3566,Universitat de València,Facultat d'Infermeria i Podologia,Grado en Podología,Sí,11.03,No,11.08,11.08,87.0,240.0,18.0,80.7
3567,Universitat de València,Facultat de Fisioteràpia,Grado en Fisioterapia,Sí,11.87,No,12.05,None,91.0,240.0,18.0,76.4
3568,Universitat de València,Facultat de Magisteri,Grado en Maestro/a en Educación Infantil,Sí,9.59,No,9.75,10.37,96.0,240.0,12.79,77.1


In [18]:
sqlGrad = "INSERT INTO Grados (id_fac, nombre, presencial, nota_corte, idioma, \
                                    nota_corte_ant, nota_admision, rendimiento, creditos, precio_creditos, \
                                    empleabilidad) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
sqlQueryFac = "SELECT id FROM Facultades WHERE id_univ = (SELECT id FROM Universidades WHERE nombre = %s) AND nombre = %s"
uniAnterior = ""
facAnterior = ""
idFac = -1
for index, row in dfNone.iterrows():
    uniActual = row['NOMBRE_UNIV']
    facActual = row['NOMBRE_CENTRO']
    if uniAnterior != uniActual or facAnterior != facActual:
        mycursor.execute(sqlQueryFac, (uniActual, facActual))
        myresult = mycursor.fetchall()
        idFac = myresult[0][0]
        uniAnterior = uniActual
        facAnterior = facActual
    if row['PRESENCIALIDAD']=="No": 
        presencialidad=0 
    else: 
        presencialidad=1
    if row['IDIOMAEXTRANJERO']=="No":
        idioma=0 
    else: 
        idioma=1
    val = (idFac, row['NOM_TITULA'], presencialidad, row['NOTA_CORTE'], 
           idioma, row['NOTA_CORTE_ANT'], row['NOTA_ADMISION'], row['RENDIMIENTO'],
           row['CREDITOS'], row['PRECIO'], row['TASA_AFIL'])
    mycursor.execute(sqlGrad, val)
mydb.commit()

In [19]:
sqlRank = "INSERT INTO Rankings (id_univ, id_campo, puntuacion) VALUES (%s, %s, %s)"
sqlQueryUniv = "SELECT id FROM Universidades WHERE nombre = %s"
for index, row in dfNoneRank.iterrows():
    mycursor.execute(sqlQueryUniv, (row['NOMBRE_UNIV'], ))
    myresult = mycursor.fetchall()
    idUni = myresult[0][0]
    val = []
    for k, v in camposDict.items():
        val.append((idUni, v, row[k]))
    mycursor.executemany(sqlRank, val)
mydb.commit()
